In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import math
import functools

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
spark

In [ ]:
clickstream_df = spark.read.options(delimiter='\t', header='True', inferSchema='True').csv('hdfs:///data/lsml/sga/clickstream.csv')

In [ ]:
# clickstream_df.show()

In [ ]:
from pyspark.sql.functions import *

In [ ]:
def shape(df):
    return df.count(), len(df.columns)

In [ ]:
# print(shape(clickstream_df))

In [ ]:
clickstream_df.dtypes

[('user_id', 'int'),
 ('session_id', 'int'),
 ('event_type', 'string'),
 ('event_page', 'string'),
 ('timestamp', 'int')]

In [ ]:
clickstream_df.registerTempTable('clickstream_tbl')

In [ ]:
zeroth_query = "select * from clickstream_tbl limit 10" # Just Checking --> Hence its named Zeroth Query

In [ ]:
# spark.sql(zeroth_query).show()

In [ ]:
first_query = "select user_id, session_id, min(timestamp) as timestamp \
                from clickstream_tbl \
                where event_type like '%error%' \
                group by user_id, session_id"

error_min_time_df = spark.sql(first_query)

In [ ]:
# print(shape(error_min_time_df))
# error_min_time_df.show(10)

In [ ]:
error_min_time_df.registerTempTable('error_min_time_tbl')

In [ ]:
second_query = "select cst.* \
               from clickstream_tbl as cst \
               left outer join \
               error_min_time_tbl as emt \
               on cst.user_id == emt.user_id and cst.session_id == emt.session_id \
               where event_type == 'page' and (emt.timestamp is null or cst.timestamp <= emt.timestamp) \
               order by cst.timestamp"

routes_df = spark.sql(second_query)

In [ ]:
# print(shape(routes_df))
# routes_df.show(10)

In [ ]:
routes_df.registerTempTable('routes_tbl')

In [ ]:
third_query = "select user_id, session_id, \
                concat_ws('-', collect_list(event_page)) as route\
                from routes_tbl \
                group by user_id, session_id"

final_routes = spark.sql(third_query)

In [ ]:
# print(shape(final_routes))
# final_routes.show()

In [ ]:
final_routes.registerTempTable('final_routes_tbl')

In [ ]:
fourth_query = "select route, count(user_id) as count \
               from final_routes_tbl \
               group by route \
               order by count desc"

user_routes = spark.sql(fourth_query)

In [ ]:
# print(shape(user_routes))
# user_routes.show()

In [ ]:
final_routes_pd = user_routes.toPandas()

In [ ]:
final_routes_pd.head(30)

,route,count
0,main,39250
1,main-tariffs,6416
2,main-news,6160
3,main-archive,5743
4,main-family,4768
5,main-digital,4135
6,main-bonus,3425
7,main-tariffs-news,1148
8,main-news-tariffs,1097
9,main-tariffs-archive,1005


In [ ]:
final_routes_pd.head(30).to_csv('bharani_lsml_sga_hiveQL.tsv', sep='\t', encoding='utf-8', header=False, index=False)

In [ ]:
! cat bharani_lsml_sga_hiveQL.tsv

main	39250
main-tariffs	6416
main-news	6160
main-archive	5743
main-family	4768
main-digital	4135
main-bonus	3425
main-tariffs-news	1148
main-news-tariffs	1097
main-tariffs-archive	1005
main-news-archive	969
main-archive-tariffs	969
main-archive-news	960
main-tariffs-family	899
main-family-tariffs	899
main-news-family	885
main-family-news	853
main-archive-family	788
main-news-digital	765
main-family-archive	748
main-tariffs-main	736
main-tariffs-digital	723
main-digital-news	720
main-digital-tariffs	705
main-spravka	693
main-archive-digital	688
main-digital-archive	666
main-news-main	655
main-tariffs-bonus	647
main-archive-main	597


In [ ]:
! curl -d "$(cat bharani_lsml_sga_hiveQL.tsv)" hadoop2-00.yandex.ru:8008/sga/task_hive

Great job! Secret keyword is 'HiveMind'
